In [ ]:
!pip3 install natsort 

In [9]:
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.utils.np_utils as np_utils
import numpy as np

from tensorflow.keras import datasets, layers, models

import natsort

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [13]:
def mySort(e):
    print(e.split(',')[0]) 
    return e.split(',')[0] 

# Load datasets 
train_set = r'/Users/gustavonrm/Desktop/VCOM/proj2/train'
test_set = []
train_classes = []


#Load train classes
file_classes = []

f = open("multiclass.csv", "r")
lines = f.readlines()

i = 0 
for line in lines:
    if i == 0: 
        i = i + 1 
        continue
    i = i + 1 
    file_classes.append(line)

f.close()
#Classes sorted to match the data order
file_classes = natsort.natsorted(file_classes)
print(file_classes[0])

#Build list only with labels 
for c in file_classes: 
    train_classes.append(c.split(',')[1].rstrip("\n"))

print(train_classes[0])
#Notmalize pixel values to be between 0 and 1
#train_set = train_set.astype('float32') / 255

def load_classes():
    
    classes = []
    #Load classees
    f = open("multiclass.csv", "r")
    lines = f.readlines()

    for line in lines:
        classes.append(line)

    f.close()

    # Sort image alphabetically to match folder order 
    i = 0; 
    for c in classes: 
        c1 = c.split(",")[0]
      
        i = i + 1
    # Remove image name 



1a0d0e5283259676,58



In [16]:
#Check data
#class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_set[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    #plt.xlabel(class_names[train_labels[i][0]])
plt.show()


TypeError: Image data of dtype <U1 cannot be converted to float